<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson OpenScale - Initial Setup

## The notebook will configure OpenScale to monitor a machine learning deployment.

### Contents

- [1.0 Install Python Packages](#setup)
- [2.0 Configure Credentials](#credentials)
- [3.0 OpenScale configuration](#openscale)
- [4.0 Create Datamart](#datamart)
- [5.0 Bind Machine Learning engines](#bind)
- [6.0 Check and setup subscriptions](#subscriptions)
- [7.0 Score the model](#score)
- [8.0 Store the variables](#store)

# 1.0 Install Python Packages <a name=setup></a>

In [1]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade ibm-ai-openscale==2.2.1 --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client-V4==1.0.55 | tail -n 1

### Action: restart the kernel!

# 2 .0 Configure credentials <a name="credentials"></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')

### The url for `WOS_CREDENTIALS` is the url of the Cloud Pak for Data cluster, i.e. `https://zen-cpd-zen.apps.com`. `username` and `password` are the credentials used to log in to the Cloud Pak for Data cluster.

In [24]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.omid-cp4d-v5-2bef1f4b4097001da9502000c44fc2b2-0001.us-south.containers.appdomain.cloud",
    "username": "<redacted>",
    "password": "<redacted>"
}

In [25]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='2.5.0'

The `DATABASE_CREDENTIALS` will be provided for you.

In [26]:
DATABASE_CREDENTIALS = {
<redacted>
}

In [27]:
SCHEMA_NAME = "NEWMANUALCONFIG"

# 3.0 Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and configure OpenScale

In [28]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [29]:
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [30]:
ai_client = APIClient4ICP(WOS_CREDENTIALS)
ai_client.version

'2.2.1'

# 4.0 Create datamart <a name="datamart"></a>

## 4.1 Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If an OpenScale datamart exists in Db2, the existing datamart will be used and no data will be overwritten.

Prior instances of the Credit model will be removed from OpenScale monitoring.

In [31]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    print('Using existing external datamart')
except:
    print('Setting up external datamart')
    ai_client.data_mart.setup(db_credentials=DATABASE_CREDENTIALS, schema=SCHEMA_NAME)

Using existing external datamart


In [32]:
data_mart_details

{'database_configuration': {'credentials': {'db': 'BLUDB',
   'db_type': 'db2',
   'hostname': 'dashdb-txn-flex-yp-dal09-168.services.dal.bluemix.net',
   'password': '<redacted>',
   'port': 50000,
   'username': 'bluadmin'},
  'database_type': 'db2',
  'location': {'schema': 'AIOSFASTPATHICP'},
  'name': 'db2'},
 'internal_database': False,
 'service_instance_crn': 'N/A',
 'status': {'state': 'active'}}

## 5.0  Bind machine learning engines <a name="bind"></a>

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. 

### Create the binding if it doesn't already exist.

In [33]:
binding_uid = None

binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance4ICP(wml_credentials=WML_CREDENTIALS))
    bindings_details = ai_client.data_mart.bindings.get_details()
binding_uid

'999'

In [34]:
ai_client.data_mart.bindings.list()

c5efdd19-0c21-4862-9949-98b23cbcf9c0,WML instance,watson_machine_learning,2020-05-20T18:50:22.394Z
ffcfd0d4-6a1a-41c2-9d45-2e84fd364a37,WML instance,watson_machine_learning,2020-05-20T17:34:02.850Z
999,ICP WML Instance,watson_machine_learning,2020-04-14T20:32:36.897Z


### 5.1 get list of assets

In [35]:
ai_client.data_mart.bindings.list_assets()

ac113f95-814f-413c-b276-3f8002f4ad7e,scottda model 6-25,2020-06-25T14:54:23.002Z,model,mllib_2.3,999,False
1859b11e-7443-4918-9737-b8f8228c20c8,sda-model-6-17-2020,2020-06-17T17:16:12.002Z,model,mllib_2.3,999,True
4ca96645-6381-47ae-8c3a-53f233d232c2,CreditRiskAutoAIExperimentv1 - P4 GradientBoostingClassifierEstimator,2020-05-20T14:23:40.002Z,model,wml-hybrid_0.1,999,False
5f0dbd06-d915-42c6-bbe5-1aeef8196552,CreditRiskSpark05192020v1,2020-05-19T14:43:19.002Z,model,mllib_2.3,999,False
c266d3bd-fea0-49af-87f0-77b4c07a5d30,samaya Risk Model May 8,2020-05-08T20:59:50.002Z,model,mllib_2.3,999,True
cd2fa3e8-520e-4ad6-91c4-e4fd401c1ec4,Demouser2Churnv1,2020-05-08T13:26:36.002Z,model,mllib_2.3,999,False
5bb357f7-0ac6-43f4-b8c1-798f7eaac787,samaya Risk Model,2020-05-06T19:00:20.002Z,model,mllib_2.3,999,True
62510199-edb8-4789-b28c-d9f92eb6ddfa,scottda model 5-6-2020,2020-05-06T15:49:51.002Z,model,mllib_2.3,999,False
62510199-edb8-4789-b28c-d9f92eb6ddfa,scottda model 5-6-2020,2020-05-06T15:49:51.002Z,model,mllib_2.3,ffcfd0d4-6a1a-41c2-9d45-2e84fd364a37,False
62510199-edb8-4789-b28c-d9f92eb6ddfa,scottda model 5-6-2020,2020-05-06T15:49:51.002Z,model,mllib_2.3,c5efdd19-0c21-4862-9949-98b23cbcf9c0,False
1b78e386-c90e-4d3c-8d6b-a3b57b1468f9,scottda model 5-3-2020,2020-05-04T17:39:21.002Z,model,mllib_2.3,999,False


###  5.2 Action: Set the MODEL_NAME to your depoloyed mllib model below:

In [36]:
MODEL_NAME = "scottda model 6-25"

In [37]:
ai_client.data_mart.bindings.get_details(binding_uid)

{'entity': {'credentials': {},
  'instance_id': '999',
  'name': 'ICP WML Instance',
  'service_type': 'watson_machine_learning',
  'status': {'state': 'active'}},
 'metadata': {'guid': '999',
  'url': '/v1/data_marts/00000000-0000-0000-0000-000000000000/service_bindings/999',
  'created_at': '2020-04-14T20:32:36.897Z'}}

## 6.0 Subscriptions <a name="subscriptions"></a>

### Only if needed, remove existing credit risk subscriptions
This code removes previous subscriptions to the Credit model to refresh the monitors with the new model and new data.
This should not be needed and is only removed to cleanup a problem situation.

In [ ]:
# subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
# for subscription in subscriptions_uids:
#    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
#    if sub_name == MODEL_NAME:
#        ai_client.data_mart.subscriptions.delete(subscription)
#        print('Deleted existing subscription for', MODEL_NAME)

###  6.1 Get the list of deployment spaces and use the GUID to set the default_space

In [38]:
wml_client.spaces.list()

------------------------------------  ----------------------  ------------------------
GUID                                  NAME                    CREATED
2fd56c70-0907-445c-9957-62ce5b1e1065  ScottDAdeploymentSpace  2020-06-17T17:05:07.749Z
d1c17f19-7b7c-4083-847e-7b0dbcee7fab  blart deployment space  2020-04-16T22:54:41.604Z
------------------------------------  ----------------------  ------------------------


### 6.2 Action: We'll use the `GUID` for your Deployment space as listed above to replace `******` for  the `default_space`  below:

In [39]:
default_space = "2fd56c70-0907-445c-9957-62ce5b1e1065"

In [40]:
wml_client.set.default_space(default_space)

'SUCCESS'

In [41]:
wml_models = wml_client.repository.get_model_details()
model_uid = None

for model_in in wml_models['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break
        
print(model_uid)

ac113f95-814f-413c-b276-3f8002f4ad7e


### 6.3 Action:  Set the DEPLOYMENT_NAME
Use the name of the Deployment that is associated with your machine learning model

In [42]:
wml_client.deployments.list()

------------------------------------  -------------------------------------------------  -----  ------------------------  -------------
GUID                                  NAME                                               STATE  CREATED                   ARTIFACT_TYPE
653f39ee-8e3b-4cde-8b38-2db5a1baf591  scottda Deployment 6-25                            ready  2020-06-25T15:00:26.190Z  model
fbbfc2cd-a364-47bc-83fe-14501e42587f  WOS-INTERNAL-9e7aa8f1-2906-4b4e-b890-8c32a5bfdda5  ready  2020-06-17T22:37:49.865Z  model
8d1e7718-a378-422e-83f0-3bcbcc2baf34  sda-deployment-6-17-2020                           ready  2020-06-17T17:17:34.826Z  model
------------------------------------  -------------------------------------------------  -----  ------------------------  -------------


In [22]:
DEPLOYMENT_NAME = "scottda Deployment 6-25"

In [43]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    print(deployment['entity']['name'])
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break
        
print(deployment_uid)

scottda Deployment 6-25
653f39ee-8e3b-4cde-8b38-2db5a1baf591


## 6.3 This code creates the model subscription in OpenScale using the Python client API. 
> Note that we need to provide the model unique identifier,and some information about the model itself.

### Check to see if subscription already exists, and use it if it does

In [44]:
subscription = None
if subscription is None:
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            print("Found existing subscription")
            subscription = ai_client.data_mart.subscriptions.get(sub)
    if subscription is None:
        print("No subscription found. Run the cell below to add the subscription")

No subscription found. Run the cell below to add the subscription


### If the subscription is not found, add it now

In [45]:
if subscription is None:
    subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(        
        model_uid,
        problem_type=ProblemType.BINARY_CLASSIFICATION,
        input_data_type=InputDataType.STRUCTURED,
        label_column='RISK',
        prediction_column='predictedLabel',
        probability_column='probability',
        feature_columns = ['CHECKINGSTATUS', 'LOANDURATION', 'CREDITHISTORY', 'LOANPURPOSE', 'LOANAMOUNT', 'EXISTINGSAVINGS', 'EMPLOYMENTDURATION',
                           'INSTALLMENTPERCENT', 'SEX', 'OTHERSONLOAN', 'CURRENTRESIDENCEDURATION', 'OWNSPROPERTY', 'AGE', 'INSTALLMENTPLANS', 'HOUSING',
                           'EXISTINGCREDITSCOUNT', 'JOB', 'DEPENDENTS', 'TELEPHONE', 'FOREIGNWORKER'],
        categorical_columns = ['CHECKINGSTATUS', 'CREDITHISTORY', 'LOANPURPOSE', 'EXISTINGSAVINGS', 'EMPLOYMENTDURATION', 'SEX', 'OTHERSONLOAN',
                               'OWNSPROPERTY', 'INSTALLMENTPLANS', 'HOUSING', 'JOB', 'TELEPHONE', 'FOREIGNWORKER']
    ))


Get subscription list

In [46]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

78338840-6e6c-4cf9-8293-a1893f607689,scottda model 6-25,model,999,2020-06-25T15:49:50.815Z
3ac0127b-fe13-4b18-9785-9aed3f211d97,sda-model-6-17-2020,model,999,2020-06-23T21:41:59.001Z
bfcf3a77-1835-47bc-a153-fe5ee16c368e,samaya Risk Model May 8,model,999,2020-05-08T21:02:50.670Z
fe732c07-f552-4156-9ca3-519ac7e51b98,samaya Risk Model,model,999,2020-05-06T19:55:08.500Z
65576d9d-ad82-47a4-9f80-be727f611c05,GermanCreditRiskModelICP,model,999,2020-04-14T20:34:31.556Z


In [47]:
subscription_details = subscription.get_details()

### 7.0 Score the model so we can configure monitors <a name="score"></a>

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [48]:
credit_risk_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        credit_risk_scoring_endpoint = deployment['entity']['status']['online_url']['url']
        
print(credit_risk_scoring_endpoint)

653f39ee-8e3b-4cde-8b38-2db5a1baf591
https://zen-cpd-zen.omid-cp4d-v5-2bef1f4b4097001da9502000c44fc2b2-0001.us-south.containers.appdomain.cloud/v4/deployments/653f39ee-8e3b-4cde-8b38-2db5a1baf591/predictions


In [49]:
fields = ['CHECKINGSTATUS', 'LOANDURATION', 'CREDITHISTORY', 'LOANPURPOSE', 'LOANAMOUNT', 'EXISTINGSAVINGS', 'EMPLOYMENTDURATION',
                           'INSTALLMENTPERCENT', 'SEX', 'OTHERSONLOAN', 'CURRENTRESIDENCEDURATION', 'OWNSPROPERTY', 'AGE', 'INSTALLMENTPLANS', 'HOUSING',
                           'EXISTINGCREDITSCOUNT', 'JOB', 'DEPENDENTS', 'TELEPHONE', 'FOREIGNWORKER']
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record scoring result: 
 fields: ['CHECKINGSTATUS', 'LOANDURATION', 'CREDITHISTORY', 'LOANPURPOSE', 'LOANAMOUNT', 'EXISTINGSAVINGS', 'EMPLOYMENTDURATION', 'INSTALLMENTPERCENT', 'SEX', 'OTHERSONLOAN', 'CURRENTRESIDENCEDURATION', 'OWNSPROPERTY', 'AGE', 'INSTALLMENTPLANS', 'HOUSING', 'EXISTINGCREDITSCOUNT', 'JOB', 'DEPENDENTS', 'TELEPHONE', 'FOREIGNWORKER', 'CHECKINGSTATUS_IX', 'CREDITHISTORY_IX', 'LOANPURPOSE_IX', 'EXISTINGSAVINGS_IX', 'EMPLOYMENTDURATION_IX', 'SEX_IX', 'OTHERSONLOAN_IX', 'OWNSPROPERTY_IX', 'INSTALLMENTPLANS_IX', 'HOUSING_IX', 'JOB_IX', 'TELEPHONE_IX', 'FOREIGNWORKER_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [20, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19], [1.0, 1.0, 1.0, 13.0, 

## 8.0 Store the variables <a name="store"></a>
### This will store the important variables for use in future notebooks

In [50]:
DEFAULT_SPACE = default_space 

%store MODEL_NAME
%store DEPLOYMENT_NAME
%store DEFAULT_SPACE
%store model_uid
%store binding_uid

Stored 'MODEL_NAME' (str)
Stored 'DEPLOYMENT_NAME' (str)
Stored 'DEFAULT_SPACE' (str)
Stored 'model_uid' (str)
Stored 'binding_uid' (str)
